In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import warnings
from lsst.rsp import get_tap_service, retrieve_query
import gc
import os

# Astropy
import astropy.units as u
from astropy.units import UnitsWarning
from astropy.timeseries import LombScargle

import matplotlib.pyplot as plt
plt.style.use('tableau-colorblind10')

from lsst.daf.butler import Butler
import lsst.daf.butler as dafButler
import lsst.afw.display as afwDisplay
import lsst.geom as geom

In [2]:
#initializing Tap and Butler
pd.set_option('display.max_rows', 20)
warnings.simplefilter("ignore", category=UnitsWarning)
service = get_tap_service()
assert service is not None
assert service.baseurl == "https://data.lsst.cloud/api/tap"

# Use lsst.afw.display with the matplotlib backend
afwDisplay.setDefaultBackend('matplotlib')

In [3]:
config = 'dp02'
collection = '2.2i/runs/DP0.2'
butler = dafButler.Butler(config, collections=collection)
skymap = butler.get('skyMap')
image_size = 100 #pixels

## explore tables, columns, and datasets available

In [4]:
#print all the available dataset types
for dt in sorted(butler.registry.queryDatasetTypes('calexp*')):
    print(dt)

DatasetType('calexp', {band, instrument, detector, physical_filter, visit_system, visit}, ExposureF)
DatasetType('calexpBackground', {band, instrument, detector, physical_filter, visit_system, visit}, Background)
DatasetType('calexpCutoutTask_config', {}, Config)
DatasetType('calexpCutoutTask_log', {band, instrument, detector, physical_filter, visit_system, visit}, ButlerLogRecords)
DatasetType('calexpCutoutTask_metadata', {band, instrument, detector, physical_filter, visit_system, visit}, PropertySet)
DatasetType('calexp_camera', {band, instrument, physical_filter, visit_system, visit}, ImageF)
DatasetType('calexp_cutouts', {band, instrument, detector, physical_filter, visit_system, visit}, Stamps)


In [5]:
query = "SELECT * FROM tap_schema.schemas"
results = service.search(query).to_table()
results

description,schema_index,schema_name,utype
str512,int32,str64,str512
Data Preview 0.1 includes five tables based on the DESC's Data Challenge 2 simulation of 300 square degrees of the wide-fast-deep LSST survey region after 5 years. All tables contain objects detected in coadded images.,2,dp01_dc2_catalogs,
"Data Preview 0.2 contains the image and catalog products of the Rubin Science Pipelines v23 processing of the DESC Data Challenge 2 simulation, which covered 300 square degrees of the wide-fast-deep LSST survey region over 5 years.",0,dp02_dc2_catalogs,
ObsCore v1.1 attributes in ObsTAP realization,1,ivoa,
A TAP-standard-mandated schema to describe tablesets in a TAP 1.1 service,100000,tap_schema,
UWS Metadata,120000,uws,


In [6]:
schema_names = results['schema_name']
for name in schema_names:
    if name.find('dp02') > -1:
        dp02_schema_name = name
        break
print("DP0.2 schema is " + dp02_schema_name)

DP0.2 schema is dp02_dc2_catalogs


In [7]:
query = "SELECT * FROM tap_schema.tables " \
        "WHERE tap_schema.tables.schema_name = '" \
        + dp02_schema_name + "' order by table_index ASC"
print(query)

results = service.search(query)
results = results.to_table()
results

SELECT * FROM tap_schema.tables WHERE tap_schema.tables.schema_name = 'dp02_dc2_catalogs' order by table_index ASC


description,schema_name,table_index,table_name,table_type,utype
str512,str512,int32,str64,str8,str512
Properties of the astronomical objects detected and measured on the deep coadded images.,dp02_dc2_catalogs,1,dp02_dc2_catalogs.Object,table,
"Properties of detections on the single-epoch visit images, performed independently of the Object detections on coadded images.",dp02_dc2_catalogs,2,dp02_dc2_catalogs.Source,table,
"Forced-photometry measurements on individual single-epoch visit images and difference images, based on and linked to the entries in the Object table. Point-source PSF photometry is performed, based on coordinates from a reference band chosen for each Object and reported in the Object.refBand column.",dp02_dc2_catalogs,3,dp02_dc2_catalogs.ForcedSource,table,
Properties of time-varying astronomical objects based on association of data from one or more spatially-related DiaSource detections on individual single-epoch difference images.,dp02_dc2_catalogs,4,dp02_dc2_catalogs.DiaObject,table,
Properties of transient-object detections on the single-epoch difference images.,dp02_dc2_catalogs,5,dp02_dc2_catalogs.DiaSource,table,
"Point-source forced-photometry measurements on individual single-epoch visit images and difference images, based on and linked to the entries in the DiaObject table.",dp02_dc2_catalogs,6,dp02_dc2_catalogs.ForcedSourceOnDiaObject,table,
"Metadata about the pointings of the DC2 simulated survey, largely associated with the boresight of the entire focal plane.",dp02_dc2_catalogs,7,dp02_dc2_catalogs.Visit,table,
Metadata about the 189 individual CCD images for each Visit in the DC2 simulated survey.,dp02_dc2_catalogs,8,dp02_dc2_catalogs.CcdVisit,table,
Static information about the subset of tracts and patches from the standard LSST skymap that apply to coadds in these catalogs,dp02_dc2_catalogs,9,dp02_dc2_catalogs.CoaddPatches,table,


In [8]:
# def get_columns_from_table(table_name):
#     results = service.search("SELECT column_name, datatype, description, unit from TAP_SCHEMA.columns "
#                          "WHERE table_name = "+table_name+" ")
#     df_table = results.to_table().to_pandas()
#     column_names = df_table['column_name'].to_list()
#     return column_names

In [9]:
results = service.search("SELECT column_name, datatype, description, unit from TAP_SCHEMA.columns "
                         "WHERE table_name = 'dp02_dc2_catalogs.CcdVisit'")
df_ccdvisit_table = results.to_table().to_pandas()
df_ccdvisit_table['column_name'].to_list()

['band',
 'ccdVisitId',
 'darkTime',
 'decl',
 'detector',
 'expMidpt',
 'expMidptMJD',
 'expTime',
 'llcdec',
 'llcra',
 'lrcdec',
 'lrcra',
 'obsStart',
 'obsStartMJD',
 'physical_filter',
 'psfSigma',
 'ra',
 'seeing',
 'skyBg',
 'skyNoise',
 'skyRotation',
 'ulcdec',
 'ulcra',
 'urcdec',
 'urcra',
 'visitId',
 'xSize',
 'ySize',
 'zenithDistance',
 'zeroPoint']

In [10]:
results = service.search("SELECT column_name, datatype, description, unit from TAP_SCHEMA.columns "
                         "WHERE table_name = 'dp02_dc2_catalogs.ForcedSource'")
df_forcedsrc_table = results.to_table().to_pandas()
df_forcedsrc_table['column_name'].to_list()

['band',
 'ccdVisitId',
 'coord_dec',
 'coord_ra',
 'detect_isPatchInner',
 'detect_isPrimary',
 'detect_isTractInner',
 'forcedSourceId',
 'localBackground_instFlux',
 'localBackground_instFluxErr',
 'localPhotoCalib',
 'localPhotoCalib_flag',
 'localPhotoCalibErr',
 'localWcs_CDMatrix_1_1',
 'localWcs_CDMatrix_1_2',
 'localWcs_CDMatrix_2_1',
 'localWcs_CDMatrix_2_2',
 'localWcs_flag',
 'objectId',
 'parentObjectId',
 'patch',
 'pixelFlags_bad',
 'pixelFlags_cr',
 'pixelFlags_crCenter',
 'pixelFlags_edge',
 'pixelFlags_interpolated',
 'pixelFlags_interpolatedCenter',
 'pixelFlags_saturated',
 'pixelFlags_saturatedCenter',
 'pixelFlags_suspect',
 'pixelFlags_suspectCenter',
 'psfDiffFlux',
 'psfDiffFlux_flag',
 'psfDiffFluxErr',
 'psfFlux',
 'psfFlux_flag',
 'psfFluxErr',
 'skymap',
 'tract']

## variable stars

In [11]:
def make_fig(cutout_image, ra, dec, out_name):
    """
    Create an image.
    should be followed with remove_figure

    Parameters
    ----------
    cutout_image : cutout_image from butler.get
    out_name : file name where you'd like to save it
    
    """
    fig = plt.figure(figsize=(4, 4))
    afw_display = afwDisplay.Display(frame=fig)
    afw_display.scale('asinh', 'zscale')
    afw_display.mtv(cutout_image.image)
    
    cutout_wcs = cutout_image.getWcs()
    radec = geom.SpherePoint(ra, dec, geom.degrees)
    xy = geom.PointI(cutout_wcs.skyToPixel(radec))
    
    afw_display.dot('x', xy.getX(), xy.getY(), size=1, ctype='orange')
    plt.gca().axis('off')
    plt.savefig(out_name)
    
    return fig
    
    
def remove_figure(fig):
    """
    Remove a figure to reduce memory footprint.

    Parameters
    ----------
    fig: matplotlib.figure.Figure
        Figure to be removed.

    Returns
    -------
    None
    """
    # get the axes and clear their images
    for ax in fig.get_axes():
        for im in ax.get_images():
            im.remove()
    fig.clf()       # clear the figure
    plt.close(fig)  # close the figure

    gc.collect()    # call the garbage collector

In [12]:
plots = []
bands = ['g']
batch_dir = './variable_stars_output'

In [13]:
if os.path.isdir(batch_dir) == False:
    os.mkdir(batch_dir)
    
if os.path.isdir(batch_dir+'/coadd_images') == False:
    os.mkdir(batch_dir+'/coadd_images')
else:
    os.system('rm -r '+batch_dir+'/coadd_images/*')

In [14]:
def query_stars(ra_deg, dec_deg, radius_deg, limit):
    #radius within which you want to query
    
    query = "SELECT mt.id_truth_type, mt.match_objectId, ts.ra, ts.dec "\
            "FROM dp02_dc2_catalogs.MatchesTruth AS mt "\
            "JOIN dp02_dc2_catalogs.TruthSummary AS ts ON mt.id_truth_type = ts.id_truth_type "\
            "WHERE ts.truth_type=2 "\
            "AND ts.is_variable = 1 "\
            "AND ts.is_pointsource = 1 "\
            "AND mt.match_objectId > 1 "\
            "AND CONTAINS(POINT('ICRS', ts.ra, ts.dec), CIRCLE('ICRS', "+ str(ra_deg)+", "+str(dec_deg)+", "+str(radius_deg)+")) <= 1 "\
            "LIMIT "+str(limit)+" "
    results = service.search(query)
    variable_stars = results.to_table().to_pandas()
    return variable_stars

In [15]:
%%time
variable_stars = query_stars(62.1, -35.8, 0.10,10)

CPU times: user 16.2 ms, sys: 4.84 ms, total: 21.1 ms
Wall time: 1.16 s


In [16]:
variable_stars

,id_truth_type,match_objectId,ra,dec
0,40966017306_2,1251261824595168123,62.400122,-44.492471
1,40966012618_2,1251261824595168965,62.433290,-44.481177
2,40969404941_2,1251261824595168905,62.498074,-44.485888
3,40966005530_2,1251261824595169442,62.554306,-44.470786
4,41020640403_2,1251261824595168496,62.541632,-44.489484
5,40901174328_2,1251261824595168050,62.582894,-44.495585
6,40745504819_2,1251261824595169490,62.511065,-44.477417
7,40555681803_2,1251261824595168058,62.375589,-44.493046
8,40901174286_2,1251261824595168554,62.571105,-44.489517
9,40745505268_2,1251261824595168630,62.530818,-44.488077


In [17]:
ccd_query = "SELECT src.ccdVisitId, src.coord_ra, src.coord_dec, src.objectId, "\
        "ccdvis.detector, ccdvis.ccdVisitId "\
        "FROM dp02_dc2_catalogs.ForcedSource AS src "\
        "JOIN dp02_dc2_catalogs.CcdVisit AS ccdvis "\
        "ON src.ccdVisitId = ccdvis.ccdVisitId "\
        "WHERE src.objectId = "+str(1252985858827510315)+" "
ccd_results = service.search(ccd_query)

In [18]:
ccd_results = ccd_results.to_table().to_pandas()

In [19]:
ccd_results

,ccdVisitId,coord_ra,coord_dec,objectId,detector,ccdVisitId2
0,904739114,70.574304,-44.59494,1252985858827510315,114,904739114
1,1006105001,70.574304,-44.59494,1252985858827510315,1,1006105001
2,1006101180,70.574304,-44.59494,1252985858827510315,180,1006101180
3,1162585058,70.574304,-44.59494,1252985858827510315,58,1162585058
4,1234500023,70.574304,-44.59494,1252985858827510315,23,1234500023
...,...,...,...,...,...,...
152,1197720145,70.574304,-44.59494,1252985858827510315,145,1197720145
153,1227927125,70.574304,-44.59494,1252985858827510315,125,1227927125
154,444114188,70.574304,-44.59494,1252985858827510315,188,444114188
155,995038159,70.574304,-44.59494,1252985858827510315,159,995038159


In [20]:
def get_cutout_image(ra_deg,dec_deg,visit, detector, band,datasetType='calexp',cutoutSideLength=image_size):
    """
    get the tract and patch of a source. currently retrieves i band only. 

    Parameters
    ----------
    ra : ra of source in degrees
    dec : dec of source in degrees
    
    """
    cutoutSize = geom.ExtentI(cutoutSideLength, cutoutSideLength)
    
    spherePoint = geom.SpherePoint(ra_deg*geom.degrees, dec_deg*geom.degrees)
    tract = skymap.findTract(spherePoint)
    patch = tract.findPatch(spherePoint)
    xy = geom.PointI(tract.getWcs().skyToPixel(spherePoint))
    bbox = geom.BoxI(xy - cutoutSize // 2, cutoutSize)
    
    my_tract = tract.getId()
    my_patch = tract.getSequentialPatchIndex(patch)
    
    parameters = {'bbox': bbox}
    dataId = {'band': band, 'tract': my_tract, 'patch': my_patch, 'visit': visit, 'detector':detector}
    cutout_image = butler.get(datasetType, dataId=dataId, parameters=parameters)
    return cutout_image

In [21]:
for index, row in variable_stars[0:1].iterrows():
    for band in ['r']:
        for visit in ccd_results['ccdVisitId'][0:2]:
            for detector in ccd_results['detector'][0:2]:
                deepCoadd= get_cutout_image(row['ra'],row['dec'],visit,detector,band,datasetType='calexp')
                figout = make_fig(deepCoadd, row['ra'],row['dec'],batch_dir+"/coadd_images/"+str(row['id_truth_type'])+\
                                  "_"+band+"_"+str(visit)+".png")
                remove_figure(figout)

LookupError: Dataset calexp with data ID {instrument: 'LSSTCam-imSim', detector: 114, visit: 904739114} could not be found in collections ('2.2i/runs/DP0.2',).